In [1]:
!pip install scikeras
!pip install split-folders # install the package to split the images in TRAIN and TEST folders
!pip install ann_visualizer # visualize NN architectures

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ann-visualizer: filename=ann_visualizer-2.5-py3-none-any.whl size=4168 sha256=c7c1e1334b91e12cdc56c9879f3c6e49558af2d4d9f49292b4a2669f2c6211d9
  Stored in directory: /root/.cache/pip/wheels/1b/fc/58/2ab1c3b30350105929308becddda4fb59b1358e54f985e1f4a
Successfully built ann-visualizer


In [7]:
# IMPORT TENSORFLOW/KERAS
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions

from keras.layers.core import Dense
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Activation
from keras.models import Sequential
from keras.layers.convolutional import MaxPooling2D
from keras.layers import BatchNormalization
from keras.layers.pooling import GlobalMaxPooling2D
from keras.preprocessing.image import ImageDataGenerator


# TRANSFER LEARNING
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50

# SCIKIT-LEARN/SCIKERAS
from sklearn.decomposition import PCA
import sklearn.metrics as metrics
from sklearn.model_selection import KFold
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# IMPORT OTHER MODULES
import os
import glob

import fnmatch # to count number of image per folder easily
import random # to select random images from a folder
import numpy as np
import pandas as pd
import splitfolders # needed to split the images in TRAIN and TEST folders
from os import mkdir
from pathlib import Path
from shutil import copyfile # to import "copyfile"
import plotly.figure_factory as ff # for printing the heatmap
from collections import defaultdict
from matplotlib import pyplot as plt
import matplotlib.image as img_mat
from sklearn.decomposition import PCA

# CV2
import cv2
from google.colab.patches import cv2_imshow 

# USE THE FOLLOWING RESOURCES

**VIDEOS**
- https://www.youtube.com/watch?v=kHtToZidh3A

GITHUB IMPLEMENTATION
- https://github.com/bnsreenu/python_for_microscopists/blob/master/176-multiclass_using_VGG_weights_PCA_NN_RF.py

Copy the notebook TRAINING e aggiungici anche questa parte di PCA all'inizio quando carichi tutte le immagini. Poi ci applichi i modelli. 
APplicare anche il datetime per il clcolo di ogni cella di modello.
Applicare anche quanto visto qua:
- https://shankarmsy.github.io/posts/pca-sklearn.html
- https://towardsdatascience.com/rgb-color-image-compression-using-principal-component-analysis-fce3f48dfdd0
- https://towardsdatascience.com/using-pca-to-reduce-number-of-parameters-in-a-neural-network-by-30x-times-fcc737159282

Unicamente per mostrare l'immagine finale sotto diverse applicazioni della PCA (con 400, 300, 200, 100, 50, 10 dimensioni), con il codice seguente:

In [ ]:
# Visualization of the learning curves
plt.figure(figsize=(15,6))

subplot_index = 1
for n_components in [400, 300, 200, 100, 50, 10]:
    
    plt.subplot(3, 2, subplot_index)
    subplot_index = subplot_index + 1

    ipca = PCA(n_components).fit(img_reshaped)
    transf_img = ipca.transform(img_reshaped)
    print("transf_img shape is:", transf_img.shape)

    print("-----------------------------------")
    print(f"Explained variances for {n_components}:", np.sum(ipca.explained_variance_ratio_ ) * 100)
    print("-----------------------------------")

    # restore the image from the subspace
    image_restored = ipca.inverse_transform(transf_img)

    # reshape the image to the original array size
    image_restored = np.reshape(image_restored, (np.size(img_data, 0),
                                                 np.size(img_data, 1),
                                                 np.size(img_data, 2)))
    
    image_restored = image_restored.astype(np.uint8)
    # cv2.imwrite(f"/content/{n_components}.jpg", cv2.cvtColor(image_restored, cv2.COLOR_RGB2BGR))

    plt.axis('off')
    plt.title('n_components:' + str(n_components))
    plt.imshow(image_restored)

plt.show()

In [ ]:
from google.colab import files
files.download('/content/100.jpg')

-------------------------------------

In [3]:
# Link Google Drive account
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
# 1.Insert the directory
import sys
sys.path.insert(0,"/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test")

In [11]:
# CONSTANT
# SCALE_PERCENT = 6 # resizing percentage
SEED_VALUE = 42
num_classes = 9 # number of output classes

# PATH TO THE DATA
DIR_TRAIN_DATA = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/train/*' # path contenente i sottoinsiemi di train, test
DIR_TEST_DATA = '/content/gdrive/MyDrive/Data_Science_2020-2022/Secondo_anno_Secondo_Semestre/FoDL_Project/Project_Example_Food/train_test/test/*' # path contenente i sottoinsiemi di train, test

In [13]:
for directory_path in glob.glob(DIR_TRAIN_DATA):
    label = directory_path.split("/")[-1]
    print(label)

french_fries
caprese_salad
pizza
greek_salad
sashimi
hot_dog
caesar_salad
hamburger
sushi


In [ ]:
SIZE = 224  #Resize images

#Capture training data and labels into respective lists
train_images = []
train_labels = [] 

# GET THE LABELS
for directory_path in glob.glob(DIR_TRAIN_DATA):
    tr_label = directory_path.split("/")[-1]
    print(label)
    # GET THE IMAGES
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)       
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(tr_label)

# CONVERT LISTS TO ARRAYS
train_images = np.array(train_images)
train_labels = np.array(train_labels)

# CAPTURE TEST/VALIDATION DATA AND LABELS INTO RESPECTIVE LISTS

test_images = []
test_labels = [] 
for directory_path in glob.glob(DIR_TEST_DATA):
    tt_label = directory_path.split("/")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(tt_label)

#Convert lists to arrays                
test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [19]:
print("Length of training images:", len(train_images))
print("Length of training labels:", len(train_labels))
print()
print("Length of training images:", len(test_images))
print("Length of training labels:", len(test_labels))

Length of training images: 1803
Length of training labels: 1803

Length of training images: 155
Length of training labels: 155


# CONTINUARE DA QUI!

In [ ]:
#Encode labels from text to integers.
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)

#Split data into test and train datasets (already split but assigning to meaningful convention)
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

###################################################################
# Scale pixel values to between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

#One hot encode y values for neural network. Not needed for Random Forest
from keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

#############################